In [2]:
import pandas as pd
import os
from datetime import datetime, timedelta
import shutil
import time
import pyperclip
import json

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import warnings


C:\Users\louy0\AppData\Local\Temp\ipykernel_2460\3077982861.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# 전역변수 정리

In [3]:
yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")
article_data_han = {}
article_data_msp = {}

## 디렉토리 정리

In [4]:
# 현재 작업 디렉토리를 가져오기
current_directory = os.getcwd()

# downloads 상대경로를 절대경로로 변환
relative_path_downloads = r'..\\..\\..\\..\\..\\Downloads'
download_path = os.path.join(current_directory, relative_path_downloads)

# "preprocessed" 상대경로를 절대경로로 변환
preprocessed_path = os.path.join(download_path, 'preprocessed')

# Check if the "preprocessed" folder exists
if os.path.exists(preprocessed_path):
    print(f"The 'preprocessed' folder already exists at: {preprocessed_path}")
else:
    # "preprocessed" 폴더 생성
    os.makedirs(preprocessed_path)
    print(f"The 'preprocessed' folder has been created at: {preprocessed_path}")
    

# ContentsList 상대경로를 절대경로로 변환
relative_path_contentslist = r'..\ContentsList.xlsx'
contentslist_path = os.path.join(current_directory, relative_path_contentslist)

# ContentStatistics 상대경로를 절대경로로 변환
relative_path_statistics = r'..\ContentsStatistics.xlsx'
statistics_path = os.path.join(current_directory, relative_path_statistics)


The 'preprocessed' folder already exists at: c:\Users\louy0\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정\Python\..\\..\\..\\..\\..\\Downloads\preprocessed


## ContentsList -> article_id -> bm, bh 구분

In [26]:
# 'ContentsList.xlsx' 파일 불러오기
df = pd.read_excel(contentslist_path)

# Filter the DataFrame to select rows where the label column starts with 'BM'
selected_rows_bm = df.loc[df['label'].str.startswith('BM')]

# Extract the article_ids from the 'node' column
extracted_values_bm = selected_rows_bm['node'].str[-12:].tolist()
article_ids_bm = [value for value in extracted_values_bm if value.isdigit()]

# Remove duplicates and sort the list
article_ids_bm = list(set(article_ids_bm))
article_ids_bm.sort()

# Filter the DataFrame to select rows where the first column starts with 'BH'
selected_rows_bh =df.loc[df['label'].str.startswith('BH')]

# Extract the article_ids from the 'node' column
extracted_values_bh = selected_rows_bh['node'].str[-12:].tolist()
article_ids_bh = [value for value in extracted_values_bh if value.isdigit()]

# Remove duplicates and sort the list
article_ids_bh = list(set(article_ids_bh))
values_to_remove = ['223203815685', '223253748515', '223267541556']
article_ids_bh = [id_bh for id_bh in article_ids_bh if id_bh not in values_to_remove]
article_ids_bm


['223203815685',
 '223208249149',
 '223209462852',
 '223212353603',
 '223215558503',
 '223229049237',
 '223240007276',
 '223241084754',
 '223247154141',
 '223250496328',
 '223253813865',
 '223258887581',
 '223261110315',
 '223264200872',
 '223267459993',
 '223267541556',
 '223270759875',
 '223273728446',
 '223278027001',
 '223282251484',
 '223283634476',
 '223286851479',
 '223289322164',
 '223291508141',
 '223293600922',
 '223297512585',
 '223298427705',
 '223299361549',
 '223300662460',
 '223304712190',
 '223306604658',
 '223320523083']

# 함수

In [6]:
# Function to get total views, likes, and comments using Selenium
def get_info(article_id, article_data):
    url = f"https://blog.stat.naver.com/blog/article/{article_id}/cv"
    browser.get(url)

    # Click necessary buttons
    # 어제 날짜 클릭
    time.sleep(1)
    browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_info_section__1U7kD > div > div:nth-child(1) > div > a").click()
    time.sleep(0.5)
     # Wait until the view counts, like counts, and comment counts are loaded
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#_root > div > div > div:nth-child(2) > div.u_ni_stats_info_wrap > div > div.u_ni_data_section > div > ul > li:nth-child(1) > strong"))
    )

    

    # Extract total views, total likes, and total comments using browser.find_element
    total_views_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(1) > strong")
    total_likes_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(2) > ul > li:nth-child(1) > strong")
    total_comments_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(2) > ul > li:nth-child(2) > strong")
    view_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div:nth-child(2) > div.u_ni_stats_info_wrap > div > div.u_ni_data_section > div > ul > li:nth-child(1) > strong")

    # Get the text values from the elements
    total_views_text = total_views_element.text.strip()
    total_likes_text = total_likes_element.text.strip()
    total_comments_text = total_comments_element.text.strip()
    view_text = view_element.text.strip()
    view_numeric = int(view_text.replace(',', ''))  # Remove commas and convert to int

    # Return the extracted data as a dictionary
    return {
        'article_id': article_id,
        'total_views': total_views_text,
        'total_likes': total_likes_text,
        'total_comments': total_comments_text,
        'view': view_numeric
    }


# MSP BLOG

In [25]:
## open Chrome -> naver -> login
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 브라우저 생성
browser = webdriver.Chrome(options=chrome_options)
browser.get('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')

try:
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#id')))
    time.sleep(3)
    nid = 'qli_2018'  # Your ID
    npw = '12341234!@'  # Your Password
    pyperclip.copy(nid)
    browser.find_element(By.CSS_SELECTOR, '#id').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)
    pyperclip.copy(npw)
    browser.find_element(By.CSS_SELECTOR, '#pw').send_keys(Keys.CONTROL + 'v')
    pyperclip.copy('')  # Clear clipboard for security
    browser.find_element(By.XPATH, '//*[@id="log.login"]').click()
    browser.get('https://admin.blog.naver.com/mysecondplace/stat/today')  # Navigate to the target page

    # Add your code here to parse the page with BeautifulSoup or perform other actions

except Exception as e:
    print(f"Exception: {str(e)}")

In [34]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def calculate_difference_and_weekday(current_year, current_month, current_day, target_year, target_month, target_day):
    current_date = datetime(current_year, current_month, current_day)
    target_date = datetime(target_year, target_month, target_day)

    # 현재 날짜와 입력된 날짜 간의 차이 계산
    delta = relativedelta(current_date, target_date)


    return delta.months

def calculate_week_and_day(target_year, target_month, target_day):
    # 입력된 날짜로 datetime 객체 생성
    input_date = datetime(target_year, target_month, target_day)

    # 입력된 날짜의 주차와 요일 계산
    week = input_date.isocalendar()[1]  # 주차
    day = input_date.weekday() + 2  # 요일 (월요일부터 1부터 시작)

    return week, day

# 현재 날짜 가져오기
current_date = datetime.now()

# 어제 날짜 계산
yesterday_date = current_date - timedelta(days=1)

# 어제 날짜의 년, 월, 일 추출
current_year = yesterday_date.year
current_month = yesterday_date.month
current_day = yesterday_date.day

# 입력 날짜
target_year = 23
target_month = 12
target_day = 1

months_difference = calculate_difference_and_weekday(current_year, current_month, current_day, target_year, target_month, target_day)
week_number, day_of_week = calculate_week_and_day(target_year, target_month, target_day)

print(f"현재로부터 {target_year}년 {target_month}월 {target_day}일은 {months_difference}달 전이며, {target_month}월의 {week}주 {day}번째 날입니다.")


# 기타 작업 및 다운로드 프로세스(다운로드 클릭, 유입분석 클릭, 날짜 선택, 다운로드 클릭)
WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_index_section__2a-c1 > a > span:nth-child(2)"))).click()
WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__1k56B > div > div > div.u_ni_option_group__JMFLr > form > ul:nth-child(2) > li:nth-child(1) > label"))).click()


#datepicker 선택
WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__2Yvfr > div > div > div.u_ni_option_group__1SvEC > div > div > div > button"))).click()

# months_difference 만큼 반복
for _ in range(int(months_difference)):
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__2Yvfr > div > div > div.u_ni_option_group__1SvEC > div > div > div > div:nth-child(4) > div > div > div.react-datepicker__month-container > div > a.react-datepicker__navigation.react-datepicker__navigation--previous.ui-datepicker-prev > span"))).click()


# XPath를 사용하여 날짜 선택
xpath_selector = f'//a[@class="react-datepicker__day--text ui-state-default" and text()="{target_day}"]'

# 해당 XPath를 사용하여 날짜를 클릭
day_element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_selector)))
day_element.click()


현재로부터 23년 12월 1일은 1달 전이며, 12월의 2주 5번째 날입니다.


In [13]:
# 빈 리스트를 생성하여 각각의 DataFrame을 저장할 준비
dfs = []

# 게시글 정보 수집
for article_id in article_ids_bm:
    metrics_data = get_info(article_id, article_data_msp)
    article_data_msp[article_id] = {
        'node': f'https://blog.naver.com/mysecondplace/{article_id}/cv', 
        'article_id': metrics_data['article_id'],
        'total_views': metrics_data['total_views'],
        'total_likes': metrics_data['total_likes'],
        'total_comments': metrics_data['total_comments'],
        'view': metrics_data['view']
    }

    # 기타 작업 및 다운로드 프로세스(다운로드 클릭, 유입분석 클릭, 날짜 선택, 다운로드 클릭)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_index_section__2a-c1 > a > span:nth-child(2)"))).click()
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__1k56B > div > div > div.u_ni_option_group__JMFLr > form > ul:nth-child(2) > li:nth-child(1) > label"))).click()
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__2Yvfr > div > div > div.u_ni_option_group__1SvEC > div > div > div > button"))).click()

    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_btn_section__3dOcL > a.u_ni_btn_download__KmUvf.u_ni_type_green"))).click()

    # 파일 다운로드 완료를 기다림
    time.sleep(5)  # 실제로는 다운로드 완료를 확인하는 더 견고한 방법을 사용하세요.
    files_in_directory = [f for f in os.listdir(download_path) if f.endswith('.xlsx')]
    
    if files_in_directory:
        latest_file = max(files_in_directory, key=lambda x: os.path.getctime(os.path.join(download_path, x)))
        new_file_name = f'{article_id}_{yesterday}.xlsx'
        os.rename(os.path.join(download_path, latest_file), os.path.join(download_path, new_file_name))
        shutil.move(os.path.join(download_path, new_file_name), os.path.join(preprocessed_path, new_file_name))

        # Construct the path to the Excel file for the current article_id
        excel_file_path = os.path.join(preprocessed_path, new_file_name)

        try:
            # Read the Excel file into a DataFrame
            df1 = pd.read_excel(excel_file_path, header=8)
            # Extract date_info from the file name and store it
            df1['date_info'] = yesterday
            df1['node'] = f'https://blog.naver.com/mysecondplace/{article_id}'  # 수정된 부분
            df1['view'] = metrics_data['view']
            # 현재의 DataFrame을 리스트에 추가
            dfs.append(df1)
        
        except Exception as e:
            print(f"Error processing file: {excel_file_path}")
            print(e)

# 모든 DataFrame을 결합하여 하나의 DataFrame으로 만듦
combined_df_msp = pd.concat(dfs, ignore_index=True)

# browser.close()


c:\Users\louy0\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\louy0\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\louy0\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\louy0\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\louy0\lab\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openp

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.225)
Stacktrace:
	GetHandleVerifier [0x00007FF779E52142+3514994]
	(No symbol) [0x00007FF779A70CE2]
	(No symbol) [0x00007FF7799176AA]
	(No symbol) [0x00007FF7798F0AFD]
	(No symbol) [0x00007FF77998CB1B]
	(No symbol) [0x00007FF7799A218F]
	(No symbol) [0x00007FF779985D93]
	(No symbol) [0x00007FF779954BDC]
	(No symbol) [0x00007FF779955C64]
	GetHandleVerifier [0x00007FF779E7E16B+3695259]
	GetHandleVerifier [0x00007FF779ED6737+4057191]
	GetHandleVerifier [0x00007FF779ECE4E3+4023827]
	GetHandleVerifier [0x00007FF779BA04F9+689705]
	(No symbol) [0x00007FF779A7C048]
	(No symbol) [0x00007FF779A78044]
	(No symbol) [0x00007FF779A781C9]
	(No symbol) [0x00007FF779A688C4]
	BaseThreadInitThunk [0x00007FFFE7147344+20]
	RtlUserThreadStart [0x00007FFFE79026B1+33]


In [14]:
combined_df_msp

NameError: name 'combined_df_msp' is not defined

In [23]:
# Rename columns
combined_df_msp.rename(columns={
    '유입경로': 'before_medium',
    '비율': 'before_medium_per',
    '상세유입경로': 'before',
    '비율.1': 'before_per'
}, inplace=True)

# Convert 'before_per' and 'before_medium_per' columns to numeric, coercing non-numeric values to NaN
combined_df_msp['before_per'] = pd.to_numeric(combined_df_msp['before_per'], errors='coerce')
combined_df_msp['before_medium_per'] = pd.to_numeric(combined_df_msp['before_medium_per'], errors='coerce')

# Round 'before_per' and 'before_medium_per' columns to 2 decimal places
combined_df_msp['before_per'] = combined_df_msp['before_per'].round(2)
combined_df_msp['before_medium_per'] = combined_df_msp['before_medium_per'].round(2)

# Add 'before_count' column
combined_df_msp['before_count'] = (combined_df_msp['view'] * (combined_df_msp['before_per'] / 100)).round(1)

# Add 'before_medium_count' column
combined_df_msp['before_medium_count'] = (combined_df_msp['view'] * (combined_df_msp['before_medium_per'] / 100)).round(1)

# Reorder columns
df_msp = combined_df_msp[['node', 'date_info', 'view', 'before', 'before_per', 'before_count', 'before_medium', 'before_medium_per', 'before_medium_count']]

# HANPRO BLOG

In [35]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 브라우저 생성
browser = webdriver.Chrome(options=chrome_options)
browser.get('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')

try:
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#id')))
    time.sleep(3)
    nid = 'hanpro911'  # Your ID
    npw = 'Zmfflwntlr1!'  # Your Password
    pyperclip.copy(nid)
    browser.find_element(By.CSS_SELECTOR, '#id').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)
    pyperclip.copy(npw)
    browser.find_element(By.CSS_SELECTOR, '#pw').send_keys(Keys.CONTROL + 'v')
    pyperclip.copy('')  # Clear clipboard for security
    browser.find_element(By.XPATH, '//*[@id="log.login"]').click()
    browser.get('https://admin.blog.naver.com/hanpro911/stat/today')  # Navigate to the target page

    # Add your code here to parse the page with BeautifulSoup or perform other actions

except Exception as e:
    print(f"Exception: {str(e)}")

In [36]:
# 빈 리스트를 생성하여 각각의 DataFrame을 저장할 준비
dfs = []

# 게시글 정보 수집
for article_id in article_ids_bh:
    metrics_data = get_info(article_id, article_data_han)
    article_data_han[article_id] = {
        'node': f'https://blog.naver.com/hanpro911/{article_id}/cv', 
        'article_id': metrics_data['article_id'],
        'total_views': metrics_data['total_views'],
        'total_likes': metrics_data['total_likes'],
        'total_comments': metrics_data['total_comments'],
        'view': metrics_data['view']
    }

    # 기타 작업 및 다운로드 프로세스(다운로드 클릭, 유입분석 클릭, 날짜 선택, 다운로드 클릭)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_index_section__2a-c1 > a > span:nth-child(2)"))).click()
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__1k56B > div > div > div.u_ni_option_group__JMFLr > form > ul:nth-child(2) > li:nth-child(1) > label"))).click()

    # WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__2Yvfr > div > div > div.u_ni_option_group__1SvEC > div > div > div > button"))).click()

    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_btn_section__3dOcL > a.u_ni_btn_download__KmUvf.u_ni_type_green"))).click()

    # 파일 다운로드 완료를 기다림
    time.sleep(5)  # 실제로는 다운로드 완료를 확인하는 더 견고한 방법을 사용하세요.
    files_in_directory = [f for f in os.listdir(download_path) if f.endswith('.xlsx')]
    
    if files_in_directory:
        latest_file = max(files_in_directory, key=lambda x: os.path.getctime(os.path.join(download_path, x)))
        new_file_name = f'{article_id}_{yesterday}.xlsx'
        os.rename(os.path.join(download_path, latest_file), os.path.join(download_path, new_file_name))
        shutil.move(os.path.join(download_path, new_file_name), os.path.join(preprocessed_path, new_file_name))

        # Construct the path to the Excel file for the current article_id
        excel_file_path = os.path.join(preprocessed_path, new_file_name)

        try:
            # Read the Excel file into a DataFrame
            df1 = pd.read_excel(excel_file_path, header=8)
            # Extract date_info from the file name and store it
            df1['date_info'] = yesterday
            df1['node'] = f'https://blog.naver.com/hanpro911/{article_id}'  # 수정된 부분
            df1['view'] = metrics_data['view']
            # 현재의 DataFrame을 리스트에 추가
            dfs.append(df1)
        
        except Exception as e:
            print(f"Error processing file: {excel_file_path}")
            print(e)

# 모든 DataFrame을 결합하여 하나의 DataFrame으로 만듦
combined_df_msp = pd.concat(dfs, ignore_index=True)

# browser.close()


In [38]:
# Rename columns
combined_df_han.rename(columns={
    '유입경로': 'before_medium',
    '비율': 'before_medium_per',
    '상세유입경로': 'before',
    '비율.1': 'before_per'
}, inplace=True)

# Convert 'before_per' and 'before_medium_per' columns to numeric, coercing non-numeric values to NaN
combined_df_han['before_per'] = pd.to_numeric(combined_df_han['before_per'], errors='coerce')
combined_df_han['before_medium_per'] = pd.to_numeric(combined_df_han['before_medium_per'], errors='coerce')

# Round 'before_per' and 'before_medium_per' columns to 2 decimal places
combined_df_han['before_per'] = combined_df_han['before_per'].round(2)
combined_df_han['before_medium_per'] = combined_df_han['before_medium_per'].round(2)

# Add 'before_count' column
combined_df_han['before_count'] = (combined_df_han['view'] * (combined_df_han['before_per'] / 100)).round(1)

# Add 'before_medium_count' column
combined_df_han['before_medium_count'] = (combined_df_han['view'] * (combined_df_han['before_medium_per'] / 100)).round(1)

# Reorder columns
df_han = combined_df_han[['node', 'date_info', 'view', 'before', 'before_per', 'before_count', 'before_medium', 'before_medium_per', 'before_medium_count']]

## Output Preprocessed Data

In [39]:
df_result = pd.concat([df_msp, df_han])

# 상대경로 지정
path_dailyrawdata = f'..\\DailyRawdata\\blog_{yesterday}.xlsx'

# 상대경로를 절대경로로 변환
file_name = os.path.abspath(os.path.join(current_directory, path_dailyrawdata))

# Create the 'DailyRawdata' directory if it doesn't exist
dailyrawdata_directory = os.path.join(current_directory, 'DailyRawdata')
os.makedirs(dailyrawdata_directory, exist_ok=True)

# Save the DataFrame to the Excel file with the constructed file name
df_result.to_excel(file_name, index=False)

# update total in statisics

In [41]:
# 기존 statistics 파일 불러오기
contentslist_path = os.path.join(current_directory, relative_path_contentslist)
df_stat = pd.read_excel(contentslist_path)

In [42]:
#article_data 모아오기
state_msp=pd.DataFrame(article_data_msp).T
state_han=pd.DataFrame(article_data_han).T
df_state= pd.concat([state_msp, state_han])

In [43]:
# df_stat과 df_state를 contents_url을 기준으로 left join 수행
df_stat_updated = pd.merge(df_stat, df_state, on='node', how='left')

# 업데이트할 컬럼 이름 설정
update_columns = ['total_views', 'total_likes', 'total_comments']

# df_stat에 컬럼이 없는 경우, df_state 값을 사용하여 컬럼을 추가하거나 업데이트
for col in update_columns:
    # 새로운 컬럼 이름 생성 (예: 'total_views_x', 'total_views_y')
    col_x = col + '_x'
    col_y = col + '_y'
    
    # df_stat_updated에 col_x가 있고, col_y도 있으면 col_x를 col_y 값으로 업데이트
    if col_x in df_stat_updated and col_y in df_stat_updated:
        df_stat_updated[col_x] = df_stat_updated.apply(lambda row: row[col_y] if pd.notna(row[col_y]) else row[col_x], axis=1)
        df_stat_updated.rename(columns={col_x: col}, inplace=True)
        df_stat_updated.drop(columns=[col_y], inplace=True)
    # df_stat_updated에 col_x만 있으면 이름만 변경
    elif col_x in df_stat_updated:
        df_stat_updated.rename(columns={col_x: col}, inplace=True)
    # df_stat_updated에 col_y만 있으면 이름만 변경
    elif col_y in df_stat_updated:
        df_stat_updated.rename(columns={col_y: col}, inplace=True)

In [44]:
df_stat_updated

,202401302,node,category,board1,board2,title,post_start,post_end,total_views,total_likes,total_comments,total_click,total_scroll,article_id,view
0,BH-0001,https://blog.naver.com/hanpro911,blog_hanpro911,main,main,"구매하기, 마이세컨플레이스 (feat. 한프로) : 네이버 블로그",20220103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BH-0002,https://blog.naver.com/hanpro911/222611549543,blog_hanpro911,야매 DIY,야매 IOT,전원주택 방범 CCTV 설치하기 #1 (1/2.. : 네이버블로그,20220103,NaN,528,3,1,NaN,NaN,222611549543,0
2,BH-0003,https://blog.naver.com/hanpro911/222611724607,blog_hanpro911,세컨하우스 개발,구매하기,세컨하우스용 시골집 매매하기 #1 (1/3) .. : 네이버블로그,20220103,NaN,"4,858",11,9,NaN,NaN,222611724607,1
3,BH-0004,https://blog.naver.com/hanpro911/222612556607,blog_hanpro911,야매 DIY,노천탕 만들기,최신 로우테크 화목 노천탕 만들기 #1 (1/.. : 네이버블로그,20220104,NaN,"1,712",5,0,NaN,NaN,222612556607,0
4,BH-0005,https://blog.naver.com/hanpro911/222612760199,blog_hanpro911,야매 DIY,노천탕 만들기,최신 로우테크 화목 노천탕 만들기 #2 (2/.. : 네이버블로그,20220104,NaN,"1,933",5,0,NaN,NaN,222612760199,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,SK-0420,웨버 바베큐그릴 47,NaN,NaN,NaN,NaN,20240130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
655,SK-0421,유가읍세컨하우스,NaN,NaN,NaN,NaN,20240130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
656,SK-0422,인천지역 세컨하우스 추천,NaN,NaN,NaN,NaN,20240130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657,SK-0423,충청도 서까래 시골집 매매,NaN,NaN,NaN,NaN,20240130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df_stat_updated.to_excel(contentslist_path)